In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'

In [ ]:
!kaggle competitions download -c deepfake-detection-challenge

 99% 4.11G/4.13G [00:32<00:00, 223MB/s]
100% 4.13G/4.13G [00:32<00:00, 136MB/s]


In [ ]:
#Mount our google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import json
import glob
import numpy as np
import cv2
import copy

video_files =  glob.glob('/content/Real videos/*.mp4')
frame_count = []

for video_file in video_files:
  cap = cv2.VideoCapture(video_file)
  if(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))<150):
    video_files.remove(video_file)
    continue
  frame_count.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))


print("frames" , frame_count)
print("Total number of videos: " , len(frame_count))
print('Average frame per video:',np.mean(frame_count))

In [ ]:
# to extract frames
def frame_extract(path):
    vidObj = cv2.VideoCapture(path)
    success = True
    while success:
        success, image = vidObj.read()
        if success:
            yield image

!pip3 install face_recognition
!mkdir '/content/drive/My Drive/DFDC_Face_only_data'

import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import face_recognition
from tqdm.autonotebook import tqdm

# process the frames
def create_face_videos(path_list, out_dir):
    already_present_count = len(glob.glob(out_dir + '*.mp4'))
    print("Number of videos already present: ", already_present_count)
    for path in tqdm(path_list):
        out_path = os.path.join(out_dir, os.path.split(path)[-1])
        file_exists = glob.glob(out_path)
        if len(file_exists) != 0:
            print("File already exists:", out_path)
            continue
        frames = []
        flag = 0
        face_all = []
        frames1 = []
        out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 30, (112, 112))
        for idx, frame in enumerate(frame_extract(path)):
            # if(idx % 3 == 0):
            if idx <= 150:
                frames.append(frame)
                if len(frames) == 4:
                    faces = face_recognition.batch_face_locations(frames)
                    for i, face in enumerate(faces):
                        if len(face) != 0:
                            top, right, bottom, left = face[0]
                        try:
                            out.write(cv2.resize(frames[i][top:bottom, left:right, :], (112, 112)))
                        except:
                            pass
                    frames = []
        try:
            del top, right, bottom, left
        except:
            pass
        out.release()


In [ ]:
create_face_videos(video_files,'/content/drive/My Drive/DFDC_Face_only_data/')